In [44]:
import psycopg2 as ps
import pandas as pd
import os
import xml.etree.ElementTree as et

In [45]:
conn_src = ps.connect(host = 'de-edu-db.chronosavant.ru',
                        port=  '5432',
                        database= 'taxi',
                        user= 'etl_tech_user',
                        password= 'etl_tech_user_password')

In [46]:
conn_tgt = ps.connect(host = 'de-edu-db.chronosavant.ru',
                        port=  '5432',
                        database= 'dwh',
                        user= 'dwh_novocherkassk',
                        password= 'dwh_novocherkassk_QbuWmGhe'
)

In [47]:
conn_src.autocommit = False
conn_tgt.autocommit = False

In [48]:
curs_src = conn_src.cursor()
curs_tgt = conn_tgt.cursor()

In [49]:
#Очищаем stg таблички
curs_tgt.execute("""delete from dwh_novocherkassk.stg_drivers""")
curs_tgt.execute("""delete from dwh_novocherkassk.stg_rides""")
curs_tgt.execute("""delete from dwh_novocherkassk.stg_movement""")
curs_tgt.execute("""delete from dwh_novocherkassk.stg_waybills""")
curs_tgt.execute("""delete from dwh_novocherkassk.stg_fact_rides""")

### 2. Захват данных из источника в STG
#для update и insert

In [50]:
#stg_drivers
curs_src.execute("""SELECT
     driver_license
,    first_name
,    last_name
,    middle_name
,    driver_valid_to
,    card_num
,    update_dt
,    birth_dt
FROM
     main.drivers
LIMIT 20""")

In [51]:
res = curs_src.fetchall()

In [52]:
names = [name[0] for name in curs_src.description]
df = pd.DataFrame(res, columns = names)

In [53]:
df

,driver_license,first_name,last_name,middle_name,driver_valid_to,card_num,update_dt,birth_dt
0,11 10 486788,Абубакр,Сысуев,Геннадиевич,2023-05-18,5278 8349 3547 8533,2022-11-18 23:50:03,1979-05-21
1,16 32 685783,Серафим,Регирер,Витальевич,2023-05-18,5136 9146 8720 3057,2022-11-18 23:51:03,1969-05-23
2,28 28 430395,Ярослав,Крицин,Теодорович,2023-05-18,4377 7335 7813 3251,2022-11-18 23:53:03,1992-11-24
3,53 75 061769,Савелий,Калинкер,Леонович,2023-05-18,5163 3164 3312 3751,2022-11-18 23:54:03,1964-06-14
4,20 78 325701,Валерий,Козоризенко,Оскарович,2023-05-18,4841 5717 8968 8376,2022-11-18 23:54:03,1964-10-26
5,61 11 627479,Сергей,Пилиневич,Османович,2023-05-18,4841 5744 2978 5461,2022-11-18 23:54:03,1987-02-12
6,86 28 968903,Захар,Рипченко,Максимович,2023-05-18,5235 2668 7497 3459,2022-11-18 23:56:03,1986-09-24
7,27 50 268996,Назар,Вашкоев,Эминович,2023-05-26,4406 6679 1727 9737,2022-11-26 00:48:03,1962-08-25
8,50 84 567141,Мансур,Туртыгин,Амирович,2023-05-18,5545 6273 4662 0728,2022-11-18 23:58:02,1985-11-08
9,49 24 017133,Яков,Зарубицкий,Стефанович,2023-05-18,4841 5738 8688 4668,2022-11-18 23:59:03,1979-10-14


In [54]:
curs_tgt.executemany("""INSERT INTO dwh_novocherkassk.stg_drivers(
	driver_license
,	first_name
,	last_name
,	middle_name
,	driver_valid_to
,	card_num
,	update_dt
,	birth_dt
)
VALUES(%s, %s, %s, %s, %s, %s, %s, %s)""", df.values.tolist())

In [55]:
#stg_rides
curs_src.execute("""SELECT
	ride_id
,	dt
,	client_phone
,	card_num
,	point_from
,	point_to
,	distance
,	price
FROM
	main.rides
LIMIT 20""")

In [56]:
res = curs_src.fetchall()

In [57]:
names = [name[0] for name in curs_src.description]
df = pd.DataFrame(res, columns = names)

In [58]:
df

,ride_id,dt,client_phone,card_num,point_from,point_to,distance,price
0,1,2022-11-18 23:50:02,+7 (908) 398-96-07,4454 3557 2601 5507,"Полярная улица, 4 к1","Бутырская улица, 65/68",13.10,196.50
1,2,2022-11-18 23:51:01,+7 (974) 226-46-16,5254 7766 4465 8236,"улица Леваневского, 24","Егорьевский проезд, 2А с3",19.88,298.20
2,3,2022-11-18 23:53:01,+7 (986) 691-11-93,4047 5738 1061 6618,"улица Лескова, 9Б","Жулебинская улица, 7",18.30,274.50
3,4,2022-11-18 23:54:02,+7 (938) 625-28-31,5136 9145 8971 5372,"Таганская улица, 5 с1","Федеративный проспект, 46 к1",5.52,82.80
4,5,2022-11-18 23:54:02,+7 (965) 889-22-07,4183 8466 8571 4336,"Сколковское шоссе, 26 к1","улица Большие Каменщики, 9 сЗ",12.94,194.10
5,6,2022-11-18 23:54:02,+7 (951) 485-44-86,5224 7011 3332 4535,"улица Космонавта Комарова, 12","улица Верхние Поля, 17 к2",18.66,279.90
6,7,2022-11-18 23:56:02,+7 (963) 906-24-86,4183 8484 7772 9688,"Заозёрная улица, 18","Лужнецкая набережная, 2/4 с29",12.87,193.05
7,8,2022-11-18 23:57:01,+7 (917) 093-12-38,4377 7315 8061 2718,"Малый Сухаревский переулок, 3 с2","3-й Красносельский переулок, 6",14.91,223.65
8,9,2022-11-18 23:58:01,+7 (950) 845-70-25,5254 7714 1092 6115,"улица Маршала Савицкого, 26 к4","Задорожная улица, 11",25.73,385.95
9,10,2022-11-18 23:59:02,+7 (981) 072-69-09,5136 9163 0387 8345,"улица Генерала Глаголева, 24 к1","1-й Вешняковский проезд, 2",3.18,47.70


In [59]:
curs_tgt.executemany("""INSERT INTO dwh_novocherkassk.stg_rides(
	ride_id
,	dt
,	client_phone
,	card_num
,	point_from
,	point_to
,	distance
,	price
)
VALUES(%s, %s, %s, %s, %s, %s, %s, %s)""", df.values.tolist())

In [60]:
#stg_movement
curs_src.execute("""SELECT
	movement_id
,	car_plate_num
,	ride
,	event
,	dt
FROM
	main.movement
LIMIT 20""")

In [61]:
res = curs_src.fetchall()

In [62]:
names = [name[0] for name in curs_src.description]
df = pd.DataFrame(res, columns = names)

In [63]:
df

,movement_id,car_plate_num,ride,event,dt
0,1,Р133ТУ199,2,READY,2022-11-18 23:55:03
1,2,Н447МУ77,3,READY,2022-11-18 23:56:03
2,3,М392УЕ777,1,READY,2022-11-18 23:57:03
3,4,М392УЕ777,1,BEGIN,2022-11-18 23:58:03
4,5,Х440КА197,5,READY,2022-11-18 23:59:03
5,6,А258КН790,6,READY,2022-11-18 23:59:03
6,7,М593МН99,4,READY,2022-11-19 00:00:03
7,8,Р133ТУ199,2,BEGIN,2022-11-19 00:01:02
8,9,Х440КА197,5,CANCEL,2022-11-19 00:01:02
9,10,А258КН790,6,BEGIN,2022-11-19 00:01:02


In [64]:
curs_tgt.executemany("""INSERT INTO dwh_novocherkassk.stg_movement(
	movement_id
,	car_plate_num
,	ride
,	"event"
,	dt
)
VALUES(%s, %s, %s, %s, %s)""", df.values.tolist())

In [65]:
#stg_waybills
df_all = pd.DataFrame(columns = ['waybill_number', 'car_num', 'license', 'start', 'stop', 'issue_dt'])

In [66]:
for file in os.listdir( '/Users/frank/Documents/LEARNING IT, Eng, עברית/IT Data Engineer/Courses/Sber - Data_Engineer/Module_Prof/project_taxi' ):
    tree = et.parse(f'/Users/frank/Documents/LEARNING IT, Eng, עברית/IT Data Engineer/Courses/Sber - Data_Engineer/Module_Prof/project_taxi/{file}')
    root = tree.getroot()
    for waybill in root.findall('waybill'):
        waybill_num = waybill.attrib['number']
        issue_dt = waybill.attrib['issuedt']
        car = waybill.find('car').text
        for driver in waybill.findall('driver'):
            license = driver.find('license').text
        for period in waybill.findall('period'):
            start = period.find('start').text
            stop = period.find('stop').text
    df = pd.DataFrame([[waybill_num, license, car, start, stop, issue_dt]], columns = ['waybill_number', 'car_num', 'license', 'license_validto', 'start', 'stop'])
    df_all = pd.concat([df, df_all])

ParseError: not well-formed (invalid token): line 1, column 0 (<string>)

In [67]:
df_all

,waybill_number,car_num,license,start,stop,issue_dt


In [68]:
curs_tgt.executemany("""INSERT INTO dwh_novocherkassk.stg_waybills(
	waybill_num
,	driver_pers_num
,	car_plate_num
,	work_start_dt
,	work_end_dt
,	issue_dt
)
VALUES(%s, %s, %s, %s, %s, %s)""", df_all.values.tolist())

### 3. Обновляем обновленные строки в хранилище

#### 3.1 Работа с dim_drivers - таблица-измерений

In [69]:
#insert новых записей
curs_tgt.execute("""INSERT INTO dwh_novocherkassk.dim_drivers(
     personnel_num
,    last_name
,    first_name
,    middle_name
,    birth_dt
,    card_num
,    driver_license_num
,    driver_license_dt
,    create_dttm
,    update_dttm
,    processed_dt
)
SELECT
     stg.last_name
,    stg.first_name
,    stg.middle_name
,    stg.birth_dt
,    stg.card_num
,    stg.driver_license
,    stg.driver_valid_to
,    stg.update_dt
,    NULL
,    now() processed_dt
FROM
     dwh_novocherkassk.stg_drivers stg
LEFT JOIN
     dwh_novocherkassk.dim_drivers t
          ON stg.driver_license = t.driver_license_num
WHERE
     t.driver_license_num IS NULL""")

OperationalError: SSL SYSCALL error: EOF detected


In [70]:
#update старых записей	
curs_tgt.execute("""UPDATE
	dwh_novocherkassk.dim_drivers tgt
SET
	last_name = tmp.last_name
,	first_name = tmp.first_name
,	middle_name = tmp.middle_name
,	birth_dt = tmp.birth_dt
,	card_num = tmp.card_num
,	driver_license_dt = tmp.driver_license_dt
,	update_dttm = tmp.update_dt
,	processed_dt = now()
FROM
	(
		SELECT
			s.last_name
		,	s.first_name
		,	s.middle_name
		,	s.birth_dt
		,	s.card_num
		,	s.driver_license_dt
		,	s.driver_license
		,	s.update_dt
		FROM
			dwh_novocherkassk.stg_drivers s
		JOIN
			dwh_novocherkassk.dim_drivers t
				ON s.driver_license = t.driver_license_num
	) tmp
WHERE
	tgt.driver_license = tmp.driver_license_num """)

InterfaceError: cursor already closed

### 4. Удаляем удаленные записи в целевой таблице

In [71]:
curs_tgt.execute("""DELETE FROM
	dwh_novocherkassk.dim_drivers
WHERE
	driver_license_num IN (
		SELECT
			target.driver_license_num
		FROM
			dwh_novocherkassk.dim_drivers tgt
		LEFT JOIN
			dwh_novocherkassk.stg_drivers stg
				ON tgt.driver_license_num = stg.driver_license
		WHERE
			stg.driver_license IS NULL
	) """)

InterfaceError: cursor already closed

### 3.2 Работа с fact_waybills - таблица-фактов

In [ ]:
#insert новых записей
curs_tgt.execute("""INSERT INTO dwh_novocherkassk.fact_waybills(
	waybill_num
,	driver_pers_num
,	car_plate_num
,	work_start_dt
,	work_end_dt
,	issue_dt
,	processed_dt
)
SELECT
	stg.waybill_num
,	stg.driver_pers_num
,	stg.car_plate_num
,	stg.work_start_dt
,	stg.work_end_dt
,	stg.issue_dt
,	now() processed_dt
FROM
	dwh_novocherkassk.stg_waybills stg
LEFT JOIN
	dwh_novocherkassk.fact_waybills t
		ON stg.waybill_num = t.waybill_num
WHERE
	t.waybill_num IS NULL""")

### 3.3 Работа с fact_rides - таблица-фактов

In [ ]:
#insert новых записей
curs_tgt.execute("""INSERT INTO dwh_novocherkassk.stg_fact_rides(
	ride_id
,	point_from_txt
,	point_to_txt
,	distance_val
,	price_amt
,	client_phone_num
,	driver_pers_num
,	car_plate_num
,	ride_arrival_dt
,	ride_start_dt
,	ride_end_dt
)
WITH tmp AS (
	SELECT
		r.ride_id
	,	r.client_phone
	,	r.point_from
	,	r.point_to
	,	r.distance
	,	r.price
	,	max(m.car_plate_num) car_plate_num
	,	max(
			CASE
				WHEN m.event = 'READY' THEN
					m.dt
			END
		) ride_arrival_dt
	,	max(
			CASE
				WHEN m.event = 'BEGIN' THEN
					m.dt
			END
		) ride_start_dt
	,	max(
			CASE
				WHEN m.event IN ('END', 'CANCEL') THEN
					m.dt
			END
		) ride_end_dt
	FROM
		dwh_novocherkassk.stg_rides r
	LEFT JOIN
		dwh_novocherkassk.stg_movement m
			ON r.ride_id = m.ride
	GROUP BY
		r.ride_id
	,	r.client_phone
	,	r.point_from
	,	r.point_to
	,	r.distance
	,	r.price
)
SELECT
	tmp.ride_id
,	tmp.point_from
,	tmp.point_to
,	tmp.distance
,	tmp.price
,	tmp.client_phone
,	w.driver_pers_num
,	tmp.car_plate_num
,	tmp.ride_arrival_dt
,	tmp.ride_start_dt
,	tmp.ride_end_dt
FROM
	tmp
LEFT JOIN
	dwh_novocherkassk.stg_waybills w
		ON tmp.car_plate_num = w.car_plate_num AND
		tmp.ride_arrival_dt BETWEEN w.work_start_dt AND w.work_end_dt
WHERE
	ride_end_dt IS NOT NULL""")

### 4. Создание отчета rep_drivers_payments

In [ ]:
curs_tgt.execute("""INSERT INTO dwh_novocherkassk.rep_drivers_payments(
	personnel_num
,	last_name
,	first_name
,	middle_name
,	card_num
,	amount
,	report_dt
)
SELECT
	dr.personnel_num
,	dr.last_name
,	dr.first_name
,	dr.middle_name
,	dr.card_num
,	sum(
		rid.price_amt - 0.2 * rid.price_amt - 47.26 * 7 * rid.distance_val / 100 - 5 * rid.distance_val
	) amount
,	(now() - '1'::interval(8))::date report_dt
FROM
	dwh_novocherkassk.dim_drivers dr
LEFT JOIN
	dwh_novocherkassk.fact_rides rid
		ON dr.driver_license_num = rid.driver_pers_num AND
		rid.ride_end_dt::date = now()::date - '1'::interval(8)
GROUP BY
	dr.personnel_num
,	dr.last_name
,	dr.first_name
,	dr.middle_name
,	distance_val
,	dr.card_num""")

In [72]:
conn_tgt.commit()

InterfaceError: connection already closed

In [73]:
conn_src.close()
conn_tgt.close()